# All the results and visualizations for the Sampta Paper

In [1]:
#Import everything
from Main.test_manifold_algorithms import test_manifold_algorithms as tma
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from Helpers.regression_helpers import *

## Checking if we have our data run

### Regression

In [2]:
reg_df = read_json_files_to_dataframe("/yunity/arusty/Graph-Manifold-Alignment/Results/GRAE_Data")
print(f"File Length: {len(reg_df)}")
reg_df.sample(5)

File Length: 1554


,method,csv_file,split,Percent_of_Anchors,Best_Params,CE,FOSCTTM,Random Forest OOB,Random Forest Emb A,Random Forest Emb B,Nearest Neighbor A,Nearest Neighbor B,Nearest Neighbor (F1 score or RMSE) A,Nearest Neighbor (F1 score or RMSE) B,Random Forest (F1 score or RMSE) A,Random Forest (F1 score or RMSE) B,GRAE,Parameter STD
1333,SPUD,hepatitis,turn,0.3,"{'knn': 11, 'OD_method': 'default', 'agg_metho...","{'42': 0.7608695652173914, '1738': 0.760869565...","{'42': 0.0, '1738': 0.0, '5271': 0.0, '9209': ...","{'42': 0.6590909090909091, '1738': 0.654545454...","{'42': 0.6428571428571429, '1738': 0.75, '5271...","{'42': 0.6428571428571429, '1738': 0.642857142...","{'42': 0.6428571428571429, '1738': 0.785714285...","{'42': 0.6785714285714286, '1738': 0.75, '5271...","{'42': 0.6502976190476192, '1738': 0.778838808...","{'42': 0.6849237718802936, '1738': 0.738095238...","{'42': 0.6483516483516484, '1738': 0.727596017...","{'42': 0.6483516483516484, '1738': 0.619047619...","{'42': {'RF-OOB': 0.6863636363636364, 'KNN_A':...","{'knn': 0.033055247335043246, 'OD_method': 0.0..."
350,NAMA,ConcreteCompressiveStrength,random,0.3,"{'OD_method': 'absolute_distance', 'agg_method...","{'42': -0.6000344677819662, '1738': -0.5407587...","{'42': 0.001600527853709115, '1738': 8.4833631...","{'42': 0.32435438176638975, '1738': 0.43688823...","{'42': 0.3061895891273244, '1738': 0.584038298...","{'42': 0.3503387634732541, '1738': 0.369002271...","{'42': 0.2077533681433419, '1738': 0.252647279...","{'42': 0.17260531468384888, '1738': 0.13704114...","{'42': 0.8552678964439436, '1738': 0.854557927...","{'42': 0.8740340124636691, '1738': 0.918277074...","{'42': 0.8003729010979516, '1738': 0.637536965...","{'42': 0.7744893443522257, '1738': 0.785222246...","{'42': {'RF-OOB': 0.3026594480490028, 'KNN_A':...","{'OD_method': 0.1717439500667326, 'agg_method'..."
826,SPUD,Cancer_Data,skewed,0.3,"{'knn': 38, 'OD_method': 'default', 'agg_metho...",{'42': 0.7644991212653779},{'42': 0.0},{'42': 0.9373626373626374},{'42': 0.9385964912280702},{'42': 0.9824561403508771},{'42': 0.8859649122807017},{'42': 0.9122807017543859},{'42': 0.8817016397202155},{'42': 0.9101214574898786},{'42': 0.9381219979737462},{'42': 0.9824561403508771},"{'42': {'RF-OOB': 0.9362637362637363, 'KNN_A':...","{'knn': 0.052198598724656015, 'OD_method': 0.0..."
219,NAMA,EnergyEfficiency,turn,0.3,"{'OD_method': 'absolute_distance', 'agg_method...","{'42': -0.27532951711560694, '1738': -0.275329...","{'42': 1.6954210069444444e-06, '1738': 1.69542...","{'42': 0.8946410565405154, '1738': 0.899791700...","{'42': 0.9040085759810895, '1738': 0.917145095...","{'42': 0.9152212379625457, '1738': 0.917934009...","{'42': 0.8792950763580957, '1738': 0.875487983...","{'42': 0.8785957248990581, '1738': 0.875022788...","{'42': 0.3515367384212206, '1738': 0.356675322...","{'42': 0.3525536507704474, '1738': 0.357340996...","{'42': 0.31349054248509506, '1738': 0.29095547...","{'42': 0.29461290163747433, '1738': 0.28956698...","{'42': {'RF-OOB': 0.8841582738716701, 'KNN_A':...","{'OD_method': 0.008727404469571542, 'agg_metho..."
990,RF-NAMA,CommunityCrime,even,0.3,"{'OD_method': 'absolute_distance', 'agg_method...",{'42': -0.2791470542231096},{'42': 1.886300828262118e-05},{'42': -0.09427154929006476},{'42': -0.01842025844995021},{'42': -0.006047580076450121},{'42': -0.015968400186989307},{'42': -0.024035841555152482},{'42': 0.9468055544570539},{'42': 0.9505572434633062},{'42': 0.9479473390431652},{'42': 0.9421714876657341},"{'42': {'RF-OOB': 0.00020467262735013314, 'KNN...",{}


In [3]:
#This helps me to see where my tests have failed
proc_reg_df = reg_df.drop(columns = ["Best_Params", "Parameter STD"])

#Average out the random seeds
for columns in ["CE", "FOSCTTM", "Random Forest OOB", "Random Forest Emb A", "Nearest Neighbor A", "Nearest Neighbor (F1 score or RMSE) A", "Random Forest (F1 score or RMSE) A",
                                                      "Random Forest Emb B", "Nearest Neighbor B", "Nearest Neighbor (F1 score or RMSE) B", "Random Forest (F1 score or RMSE) B"]:
    proc_reg_df[columns] = [np.mean(list(d.values())) for d in proc_reg_df[columns].values]

proc_reg_df["Combined_Metric"] = proc_reg_df["CE"] - proc_reg_df["FOSCTTM"]

baselines = pd.read_csv("/yunity/arusty/Graph-Manifold-Alignment/Results/RegressionData/BaselineDataTrainTest.csv", index_col=0).rename(columns = {"KNN_BL_A": "A_Classification_Score", "KNN_BL_B": "B_Classification_Score"})
baselines_grouped = baselines.groupby(by = ["csv_file", "split"]).max().drop(columns = ["KNN"]).reset_index()

# Group by csv_file and select the row with the 3rd smallest knn value
third_smallest_knn = baselines.groupby(by = ["csv_file", "split"]).apply(lambda x: x.nsmallest(3, 'KNN').iloc[-1]).reset_index(drop=True)

# Ensure the resulting DataFrame has the same format as baselines_grouped
third_smallest_knn_grouped = third_smallest_knn.groupby(by=["csv_file", "split"]).agg({
    "A_Classification_Score": "max",
    "B_Classification_Score": "max"
}).reset_index()
proc_reg_df = proc_reg_df.merge(third_smallest_knn_grouped, how = "left")


reg_results_df = plt_methods_by_CSV_max(df = proc_reg_df, metric = "Combined_Metric", return_df=True)#.head(13) #, split = "skewed"
reg_results_df

,csv_file,SSMA,MAGAN,DTA,SPUD,MASH,MASH-,RF-MASH-,NAMA,RF-NAMA,PCR,JLMA,MASH_RF,MALI_RF,MALI,SPUD_RF,BL_A,BL_B
0,ConcreteCompressiveStrength,0.579510,NaN,0.900176,0.355952,0.640715,0.769619,0.774758,-0.159360,-0.179408,0.900512,0.845670,NaN,-0.355119,-0.128310,0.500979,0.610397,0.715961
1,hill_valley,NaN,NaN,NaN,0.546205,0.696134,0.691128,0.744248,0.570952,0.544554,NaN,0.279086,0.367250,NaN,NaN,0.674836,NaN,NaN
2,CommunityCrime,0.497917,NaN,0.793918,0.535578,0.710046,0.712440,0.090362,-0.142984,-0.208392,0.786104,0.740419,NaN,-0.290245,-0.139376,0.535960,0.592101,0.599702
3,heart_disease,NaN,NaN,NaN,0.626263,0.717172,0.717172,0.546926,0.515140,0.577778,NaN,0.705587,0.663362,NaN,NaN,0.657228,NaN,NaN
4,Automobile,0.757670,NaN,0.882649,0.715318,0.767879,0.821432,0.758224,0.343892,0.025612,0.881048,0.804208,NaN,-0.139808,0.795496,0.856611,0.624074,0.717398
5,audiology,NaN,NaN,NaN,0.714571,0.775654,0.753457,0.725285,0.504321,0.321053,NaN,0.805604,0.668338,NaN,NaN,0.747368,NaN,NaN
6,heart_failure,NaN,NaN,NaN,0.749164,0.779698,0.770797,0.710418,0.658863,0.745819,NaN,0.773136,0.724074,NaN,NaN,0.889632,NaN,NaN
7,ConcreteSlumpTest,0.513335,NaN,0.890893,0.623485,0.780732,0.751162,0.634921,0.123935,-0.034983,0.816872,0.875523,NaN,-0.171756,0.852770,0.766478,0.745238,0.779765
8,hepatitis,NaN,NaN,NaN,0.760870,0.780923,0.789388,0.653340,0.695652,0.569544,NaN,0.758580,0.721219,NaN,NaN,0.801449,NaN,NaN
9,diabetes,NaN,NaN,NaN,0.734635,0.788281,0.790365,0.740346,0.666656,0.684633,NaN,0.802577,0.714581,NaN,NaN,0.763021,NaN,NaN
